# Prueba de controlador de motor en C++ -rotateTo()

**Limitación**: no maneja bien el cambiar a velocidad contraria cuando se pasa del setpoint. Revisar esa parte.

In [91]:
from plotutils import plot_motor_log

In [100]:
MOTOR_LOG_CSV_FILENAME="cpp/motor_controller/build/manual_control_test.csv"
df = pd.read_csv(MOTOR_LOG_CSV_FILENAME)
df.head()

,t,pos_deg,vel_deg_s,mode,scan_state,scan_initial,scan_end,manual_target,throttle,cmd
0,0.00,1--167.325,2-0,3-2,4-0,5-0,6-100,7-200,8-0,9-0
1,0.01,1--167.325,2-0,3-2,4-0,5-0,6-100,7-200,8-0,9-0
2,0.02,1--167.325,2-0,3-2,4-0,5-0,6-100,7-200,8-0,9-0
3,0.03,1--167.325,2-0,3-2,4-0,5-0,6-100,7-200,8-0,9-0
4,0.04,1--167.325,2-0,3-2,4-0,5-0,6-100,7-200,8-0,9-0


In [93]:
plot_motor_log(df,mode="scan")

,t,pos_deg,vel_deg_s,mode,scan_state,scan_initial,scan_end,manual_target,throttle,cmd,position_unwrapped_deg
0,0.00,167.325,0.000000,2,0,0,100,200,0.0,0,167.325
1,0.01,167.325,-0.057296,2,0,0,100,200,-1.0,6,167.325
2,0.02,167.324,-0.114506,2,0,0,100,200,-1.0,6,167.324
3,0.03,167.323,-0.171630,2,0,0,100,200,-1.0,6,167.323
4,0.04,167.321,-0.228668,2,0,0,100,200,-1.0,6,167.321
